<a href="https://colab.research.google.com/github/DmitryKutsev/NIS_SentiFrame/blob/master/fasttext_research.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd

In [0]:
senti_df = pd.read_csv(\
    'https://raw.githubusercontent.com/DmitryKutsev/NIS_SentiFrame/master/annotations/SENTIFRAME%20-%20case_annotation.csv')

In [7]:
senti_df

,verb,polar_or_not,polarity_dima,polarity_ira,polarity_masha,"падеж первого аргумента, по умолчанию nomn","падеж второго аргумента в нотации pymorphy2, по умолчанию accs",Комментарий,"""Класс""","rel(А0,A1)","rel(А1,A0)"
0,арестовать,6,0,0,0,NaN,accs,NaN,NaN,NaN,NaN
1,арестовывать,6,0,0,0,NaN,accs,NaN,NaN,NaN,NaN
2,атаковать,6,0,0,0,NaN,accs,NaN,NaN,NaN,NaN
3,беречь,3,1,1,1,NaN,accs,NaN,NaN,NaN,NaN
4,беспокоить,6,0,0,0,NaN,accs,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
440,шантажировать,6,0,0,0,NaN,accs,NaN,NaN,NaN,NaN
441,шокировать,3,0,0,0,NaN,accs,NaN,NaN,NaN,NaN
442,штурмовать,6,0,0,0,NaN,accs,NaN,NaN,NaN,NaN
443,эксплуатировать,3,0,0,0,NaN,accs,NaN,NaN,NaN,NaN


In [0]:
senti_df['polarity_summ'] = senti_df['polarity_masha'] + senti_df['polarity_dima'] + senti_df['polarity_ira']

In [0]:
my_senti_df = senti_df.drop(['polarity_dima', 'polarity_masha', 'polarity_ira', 'падеж первого аргумента, по умолчанию nomn', '"Класс"','rel(А0,A1)','rel(А1,A0)'], axis=1)

In [0]:
my_senti_df['падеж второго аргумента в нотации pymorphy2'] = my_senti_df['падеж второго аргумента в нотации pymorphy2, по умолчанию accs']

In [24]:
my_senti_df = my_senti_df.drop(['падеж второго аргумента в нотации pymorphy2, по умолчанию accs'], axis=1)
my_senti_df = my_senti_df.drop(['Комментарий'], axis=1)
my_senti_df

,verb,polar_or_not,polarity_summ,падеж второго аргумента в нотации pymorphy2
0,арестовать,6,0,accs
1,арестовывать,6,0,accs
2,атаковать,6,0,accs
3,беречь,3,3,accs
4,беспокоить,6,0,accs
...,...,...,...,...
440,шантажировать,6,0,accs
441,шокировать,3,0,accs
442,штурмовать,6,0,accs
443,эксплуатировать,3,0,accs


In [25]:
#tayga_none_fasttextcbow_300_10_2019

!wget http://vectors.nlpl.eu/repository/20/187.zip

--2020-05-15 11:07:06--  http://vectors.nlpl.eu/repository/20/187.zip
Resolving vectors.nlpl.eu (vectors.nlpl.eu)... 129.240.189.225
Connecting to vectors.nlpl.eu (vectors.nlpl.eu)|129.240.189.225|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2692389554 (2.5G) [application/zip]
Saving to: ‘187.zip’

187.zip             100%[===================>]   2.51G  9.48MB/s    in 4m 43s  

2020-05-15 11:11:51 (9.07 MB/s) - ‘187.zip’ saved [2692389554/2692389554]



In [26]:
!wget https://raw.githubusercontent.com/nicolay-r/RuSentiFrames/master/collection.json

--2020-05-15 11:11:52--  https://raw.githubusercontent.com/nicolay-r/RuSentiFrames/master/collection.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 674938 (659K) [text/plain]
Saving to: ‘collection.json’

collection.json     100%[===================>] 659.12K  --.-KB/s    in 0.08s   

2020-05-15 11:11:53 (8.11 MB/s) - ‘collection.json’ saved [674938/674938]



In [27]:
!pip install pymorphy2[fast]

     |████████████████████████████████| 51kB 1.7MB/s 
     |████████████████████████████████| 7.1MB 6.2MB/s 
     |████████████████████████████████| 378kB 45.5MB/s 
  Created wheel for DAWG: filename=DAWG-0.8.0-cp36-cp36m-linux_x86_64.whl size=853486 sha256=33c5523bbbf2a29862906f2bca2d1dfce18e91f67f4c3eedc1c24ebaa5d984b7
  Stored in directory: /root/.cache/pip/wheels/3d/1f/f0/a5b1f9d02e193c997d252c33d215f24dfd7a448bc0166b2a12
Successfully built DAWG


In [28]:
!unzip 187.zip

Archive:  187.zip
  inflating: meta.json               
  inflating: model.model             
  inflating: model.model.vectors_ngrams.npy  
  inflating: model.model.vectors.npy  
  inflating: model.model.vectors_vocab.npy  
  inflating: README                  


In [0]:
from collections import Counter
import json
import numpy as np
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity
import pymorphy2
import gensim
morph = pymorphy2.MorphAnalyzer()

In [0]:
positive_multi_seed = ['одобрять', 'хвалить', 'поощрять', 'любить']
negative_multi_seed = ['ненавидеть', 'убить', 'ругать', 'злиться']
positive_single_seed = ['ценить']
negative_single_seed = ['ненавидеть']

In [33]:
fasttext_model = gensim.models.KeyedVectors.load("model.model")

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
def make_seed_embedding(seed_dict, model):
  '''
  Принимает на вход список, на выходе дает средний вектор всех слов из этого списка
  '''
  summ = 0
  for verb in seed_dict:
    summ = summ + model.get_vector(verb)
  vector = summ/len(seed_dict)

  return vector

positive_multi_embedding = make_seed_embedding(positive_multi_seed, fasttext_model)
negative_multi_embedding = make_seed_embedding(negative_multi_seed, fasttext_model)

In [0]:
axis = positive_multi_embedding  - negative_multi_embedding

In [62]:
cosine_similarity(np.atleast_2d(fasttext_model.get_vector('беречь')), np.atleast_2d(axis))[0][0]

0.055182453

In [63]:
cosine_similarity(np.atleast_2d(fasttext_model.get_vector('арестовывать')), np.atleast_2d(axis))[0][0]

-0.047116164

In [0]:
fasttext_pol = []
cosine_lst = []
manual_pol = []
treshold = 0.03
for i in range(len(my_senti_df['verb'])):
  cosine = cosine_similarity(np.atleast_2d(fasttext_model.get_vector(my_senti_df['verb'][i])), np.atleast_2d(axis))
  if cosine >= treshold:
    fasttext_pol.append(1)
  else:
    fasttext_pol.append(0)
  cosine_lst.append(cosine[0][0])
  if my_senti_df['polarity_summ'][i] > 1:
    manual_pol.append(1)
  else:
    manual_pol.append(0)
my_senti_df['fasttext_polarity'] = fasttext_pol
my_senti_df['manual_polarity'] = manual_pol
my_senti_df['cosine_distance'] = cosine_lst

  

In [96]:
my_senti_df

,verb,polar_or_not,polarity_summ,падеж второго аргумента в нотации pymorphy2,fasttext_polarity,manual_polarity,cosine_distance
0,арестовать,6,0,accs,0,0,-0.045196
1,арестовывать,6,0,accs,0,0,-0.047116
2,атаковать,6,0,accs,0,0,-0.170026
3,беречь,3,3,accs,1,1,0.055182
4,беспокоить,6,0,accs,0,0,-0.252901
...,...,...,...,...,...,...,...
440,шантажировать,6,0,accs,0,0,-0.049897
441,шокировать,3,0,accs,0,0,-0.043513
442,штурмовать,6,0,accs,0,0,-0.095362
443,эксплуатировать,3,0,accs,1,0,0.134053


In [0]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score, roc_curve, accuracy_score, f1_score, precision_score, recall_score
from matplotlib import pyplot

In [98]:
my_senti_df['fasttext_polarity'].value_counts()

0    333
1    112
Name: fasttext_polarity, dtype: int64

In [99]:
my_senti_df['manual_polarity'].value_counts()

0    322
1    123
Name: manual_polarity, dtype: int64

In [104]:
acc = accuracy_score(my_senti_df['manual_polarity'], my_senti_df['fasttext_polarity'])
micro_f1 = f1_score(my_senti_df['manual_polarity'], my_senti_df['fasttext_polarity'], average = None)
micro_p = precision_score(my_senti_df['manual_polarity'], my_senti_df['fasttext_polarity'], average = None)
micro_r = recall_score(my_senti_df['manual_polarity'], my_senti_df['fasttext_polarity'], average = None)
print('acc={0:1.4f}'.format(acc))
print('F1={0:1.4f}, precision={1:1.4f}, recall={2:1.4f}'.format(micro_f1[-1], micro_p[-1], micro_r[-1]))

acc=0.8045
F1=0.6298, precision=0.6607, recall=0.6016
